In [1]:
import urllib as u
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

api_key='64d94994afb9689c49b660ebe9dee39c'

In [2]:
# Function to read in json data from FMP
def get_json_data(url):
    '''
    Scrape data (which must be json format) from given url
    Input: url to financialmodelingprep API
    Output: json file
    '''
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

In [3]:
# Function to get sector of a Stock from its profile json file
def find_sector(json):
    
    for k, v in a.items():
        if k == 'symbol':
            symbol=v

    for k, v in a.items():
        if k == 'profile':
            profile=v

    for k, v in profile.items():
        if k == 'sector':
            sector=v
    return(symbol, sector)

In [4]:
# Function to get Year 2019 Financial Indicators from json file
def find_fin_indicators(json, indicator):
    cat = ['financials','growth','metrics']
    
    for k, v in json.items():
        if k == 'symbol':
            symbol=v

    for k, v in json.items():
        if k in cat:
            category = v

    indicators_2019 = [[(k,v) for k,v in d.items()] for d in category if d['date'][:4] == '2019'] # Generator creates a list of lists
    indicators_2019_list = [item for sublist in indicators_2019 for item in sublist] # Creating a flat list

    a = []

    for tup in (x for x in indicators_2019_list if x[0] in indicator):
        a.append((symbol,) + tup)
    return a

In [5]:
# Function to get Year 2019 Ratios from json file
def find_ratios_indicators(json, indicator):
    global ratios, symbol
    
    for k in json.items():
        symbol = json['symbol']
    
    for k in json.items():
        ratios = json['ratios']
            
    ratios_2019 = [d for d in ratios if d['date'][:4] == '2019']
    
    unpack1 = ratios_2019[0]['investmentValuationRatios']
    unpack2 = ratios_2019[0]['profitabilityIndicatorRatios']
    unpack3 = ratios_2019[0]['operatingPerformanceRatios']
    unpack4 = ratios_2019[0]['liquidityMeasurementRatios']
    unpack5 = ratios_2019[0]['debtRatios']
    unpack6 = ratios_2019[0]['cashFlowIndicatorRatios']
    
    unpack = unpack1
    unpack.update(unpack2)
    unpack.update(unpack3)
    unpack.update(unpack4)
    unpack.update(unpack5)
    unpack.update(unpack6)
    
    full_set = []
    reduced_set =[]

    for row in unpack.items():
            full_set.append(row)
    
    for tup in (x for x in full_set if x[0] in indicator):
        reduced_set.append((symbol,) +tup)
    
    return reduced_set

In [8]:
# Full list of available stocks from FMP
url = 'https://financialmodelingprep.com/api/v3/stock/list?apikey=' + api_key
ticks_json = get_json_data(url)

stocks = pd.DataFrame(ticks_json)
stocks = stocks[['symbol','name','exchange']]
stocks = stocks.dropna(how='any',subset=['symbol','name','exchange'])

In [12]:
# Limit the analysis to stocks of the New York Stock Exchange and Nasdaq Global
stocks_nyse = stocks[(stocks['exchange'] == 'Nasdaq Global Select') | (stocks['exchange'] == 'New York Stock Exchange')]
stocks_nyse.reset_index(drop=True, inplace=True)
tickers_nyse = stocks_nyse['symbol'].tolist()

In [13]:
# For Testing (Do not run cell above!)
tickers_nyse = ['AAPL','WMT']

In [14]:
# Get sectors from company profiles
tickers_nyse_sector = []

for tick in tqdm(tickers_nyse):
    url = 'https://financialmodelingprep.com/api/v3/company/profile/' + tick + '?apikey=' + api_key 
    a = get_json_data(url)
    tickers_nyse_sector.append(find_sector(a))

100%|██████████| 2/2 [00:01<00:00,  1.57it/s]


In [15]:
tickers_nyse_sector = pd.DataFrame(tickers_nyse_sector)
tickers_nyse_sector.columns = ['symbol', 'sector']

stocks_nyse = stocks_nyse.merge(tickers_nyse_sector)

In [16]:
# Load financial indicators from the provided .txt file
indicators = []
filename = 'indicators.txt'
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

In [17]:
# Get all indicators
inc_statement_indicators = []
cash_flow_indicators = []
bs_indicators = []
fs_growth_indicators = []
metrics_indicators = []
ratios_indicators = []

for tick in tqdm(tickers_nyse):
    try:
        url0 = 'https://financialmodelingprep.com/api/v3/financials/income-statement/' + tick + '?apikey=' + api_key
        url1 = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/' + tick + '?apikey=' + api_key
        url2 = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/' + tick + '?apikey=' + api_key
        url3 = 'https://financialmodelingprep.com/api/v3/financial-statement-growth/' + tick + '?apikey=' + api_key
        url4 = 'https://financialmodelingprep.com/api/v3/company-key-metrics/' + tick + '?apikey=' + api_key
        url5 = 'https://financialmodelingprep.com/api/v3/financial-ratios/' + tick + '?apikey=' + api_key
        
        a0 = get_json_data(url0)
        a1 = get_json_data(url1)
        a2 = get_json_data(url2)
        a3 = get_json_data(url3)
        a4 = get_json_data(url4)
        a5 = get_json_data(url5)
    except:
        pass
    
    inc_statement_indicators.append(find_fin_indicators(a0,indicators))
    cash_flow_indicators.append(find_fin_indicators(a1,indicators))
    bs_indicators.append(find_fin_indicators(a2,indicators))
    fs_growth_indicators.append(find_fin_indicators(a3,indicators))
    metrics_indicators.append(find_fin_indicators(a4,indicators))
    ratios_indicators.append(find_ratios_indicators(a5,indicators))
    
    fin_indicators = inc_statement_indicators + cash_flow_indicators + bs_indicators + fs_growth_indicators + metrics_indicators + ratios_indicators
    fin_indicators_list = [item for sublist in fin_indicators for item in sublist]

100%|██████████| 2/2 [00:10<00:00,  5.26s/it]


In [18]:
# Transform the list of indicators in a DataFrame
temp = []
for j in fin_indicators_list:
    temp.append(j[1])
    df = pd.DataFrame(temp)
    df.columns = ['variable']
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
temp2 = []
for i in tickers_nyse:
    for j in fin_indicators_list:
        if j[0] == i:
            temp2.append(j[2])
            df_temp = pd.DataFrame(temp2)
            df_temp.columns = [j[0]]
    df = df.merge(df_temp, left_index = True, right_index = True)
    del df_temp
    temp2.clear()

In [43]:
df

,variable,AAPL,WMT
0,Revenue Growth,-0.0204107758053,0.028104720162
1,EBIT Margin,0.245720171885,0.0426842662882
2,Net Profit Margin,0.212380945059845,0.0129664369514293
3,Investing Cash flow,45896000000.0,-24036000000.0
4,Dividend payments,-14119000000.0,-6102000000.0
5,Total assets,338516000000.0,219295000000.0
6,Total liabilities,248028000000.0,139661000000.0
7,Net Income Growth,-0.0718113251919,-0.323666599067
8,Debt Growth,-0.0562179537573,0.296644467213
9,Debt to Equity,2.7410043320661304,1.9264649084087397


In [25]:
# Set Symbol as Index in Stocks List
stocks_nyse_labelled = stocks_nyse.set_index('symbol')
stocks_nyse_labelled

,name,exchange,sector
symbol,,,
AAPL,Apple Inc,Nasdaq Global Select,Technology
WMT,Walmart Inc,New York Stock Exchange,Consumer Defensive


In [61]:
# Set symbol as Index in Dataframe with Indicators
df_labelled = df.set_index('variable')
df_labelled_trasposed = df_labelled.T
df_labelled_trasposed = df_labelled_trasposed.reset_index()
data = df_labelled_trasposed.rename(columns={'index': 'symbol'})
data = data.set_index('symbol')
data

variable,Revenue Growth,EBIT Margin,Net Profit Margin,Investing Cash flow,Dividend payments,Total assets,Total liabilities,Net Income Growth,Debt Growth,Debt to Equity,Payout Ratio,returnOnEquity,assetTurnover
symbol,,,,,,,,,,,,,
AAPL,-0.0204107758053,0.245720171885,0.212380945059845,45896000000.0,-14119000000.0,338516000000.0,248028000000.0,-0.0718113251919,-0.0562179537573,2.7410043320661304,0.25551976255972203,0.6106445053487756,0.7685722388306608
WMT,0.028104720162,0.0426842662882,0.0129664369514293,-24036000000.0,-6102000000.0,219295000000.0,139661000000.0,-0.323666599067,0.296644467213,1.9264649084087397,0.9148425787106447,0.09200507614213198,2.345721516678447
